In [113]:
import os
from deta import Deta
from dotenv import load_dotenv
import sqlite3
import pandas as pd
import numpy as np

### Loading DB key

In [2]:
load_dotenv(".env")
DETA_KEY = os.getenv("DETA_KEY")

### Initializing DB key

In [3]:
deta = Deta(DETA_KEY)

### Initializing DB

In [4]:
db = deta.Base("exp_tracker_app")

### DB functions

In [5]:
def insert_period(period, incomes, expenses, comment):
    """Returns monthly report"""
    return db.put({"key": period, "incomes": incomes, "expenses": expenses, "comment": comment})

def fetch_all_periods():
    """Returns a dict of all periods"""
    res = db.fetch()
    return res.items

def get_period(period):
    return db.get(period)

### Example data

In [6]:
period = "2022_May"
comment = "init"
incomes = {'Salary': 1500, 'Dividend': 500, 'Invest Return': 100}
expenses = {'Food': 200, 'Loan': 300, 'CC Bill': 30}

#### DB inserting example data

In [7]:
insert_period(period, incomes, expenses, comment)

{'comment': 'init',
 'expenses': {'CC Bill': 30, 'Food': 200, 'Loan': 300},
 'incomes': {'Dividend': 500, 'Invest Return': 100, 'Salary': 1500},
 'key': '2022_May'}

#### Fetching all data from DB

In [8]:
fetch_all_periods()

[{'comment': 'init',
  'expenses': {'CC Bill': 30, 'Food': 200, 'Loan': 300},
  'incomes': {'Dividend': 500, 'Invest Return': 100, 'Salary': 1500},
  'key': '2022_May'},
 {'comment': 'first entry',
  'expenses': {'CC Bill': 30, 'Food': 40, 'Loan': 170, 'Others': 200},
  'incomes': {'Dividend': 10, 'Invest Return': 20, 'Salary': 450},
  'key': '2023_January'}]

#### Fetching single period data from DB

In [9]:
get_period("2022_May")

{'comment': 'init',
 'expenses': {'CC Bill': 30, 'Food': 200, 'Loan': 300},
 'incomes': {'Dividend': 500, 'Invest Return': 100, 'Salary': 1500},
 'key': '2022_May'}

In [122]:
db2 = deta.Base("wallet_db")

In [40]:
def insert_trans(period, date, _type, amount, wallet_id, category_id, transfer_wallet_id, trans_amount, subcategory_id, note):
    """Inserts daily transaction"""
    return db2.put({'key': period, 'date': date, 'type': _type, 'amount': amount, 'wallet_id': wallet_id, 'category_id': category_id, 'transfer_wallet_id': transfer_wallet_id, 'trans_amount': trans_amount, 'subcategory_id': subcategory_id, 'note': note})

def fetch_all_trans():
    """Returns a dict of all periods"""
    res = db2.fetch()
    return res.items

# def get_period(period):
#     """Returns monthly report"""
#     return db.get(period)

In [39]:
insert_trans('Jun-23', '24/06/2023', 'inc', 10, 1, 2, -1, 0, 0, 'test')

{'amount': 10,
 'category_id': 2,
 'date': '24/06/2023',
 'key': 'Jun-23',
 'note': 'test',
 'subcategory_id': 0,
 'trans_amount': 0,
 'transfer_wallet_id': -1,
 'type': 'inc',
 'wallet_id': 1}

In [128]:
fetch_all_trans()

[{'amount': -17.5,
  'category_id': 6,
  'date': '2021-04-30',
  'key': 'Apr_2021',
  'note': 'to dk',
  'subcategory_id': 0,
  'trans_amount': 0,
  'transfer_wallet_id': -1,
  'type': 'exp',
  'wallet_id': 9},
 {'amount': 150,
  'category_id': 39,
  'date': '2022-04-30',
  'key': 'Apr_2022',
  'note': 'from sahil - for facewash',
  'subcategory_id': 0,
  'trans_amount': 0,
  'transfer_wallet_id': -1,
  'type': 'inc',
  'wallet_id': 4},
 {'amount': -100,
  'category_id': 6,
  'date': '2023-04-28',
  'key': 'Apr_2023',
  'note': 'me+kaundal - juice',
  'subcategory_id': 0,
  'trans_amount': 0,
  'transfer_wallet_id': -1,
  'type': 'exp',
  'wallet_id': 2},
 {'amount': -49,
  'category_id': 32,
  'date': '2020-08-07',
  'key': 'Aug_2020',
  'note': 'mama chhoto',
  'subcategory_id': 0,
  'trans_amount': 0,
  'transfer_wallet_id': -1,
  'type': 'exp',
  'wallet_id': 4},
 {'amount': 100,
  'category_id': 0,
  'date': '2021-08-31',
  'key': 'Aug_2021',
  'note': 'by sakshi',
  'subcategory_

In [50]:
dbfile = 'ignore/wallet-database.db'
con = sqlite3.connect(dbfile)
cur = con.cursor()
transaction_df = pd.read_sql_query('SELECT * FROM trans', con)          #? all transactions table
con.close()
df = transaction_df.copy()
df['amount'] = df['amount'].div(100).round(2)
df['trans_amount'] = df['trans_amount'].div(100).round(2)
df['type'] = df['type'].replace([0, 1, 2], ['inc', 'exp', 'tran'])
df.drop(['id', 'fee_id', 'account_id', 'debt_id', 'debt_trans_id', 'memo'], axis = 1, inplace = True)
df['date_time'] = pd.to_datetime(df["date_time"], unit='ms').dt.date
df = df[['date_time', 'type', 'amount', 'wallet_id', 'category_id', 'transfer_wallet_id', 'trans_amount', 'subcategory_id', 'note']]
df

,date_time,type,amount,wallet_id,category_id,transfer_wallet_id,trans_amount,subcategory_id,note
0,2020-08-31,exp,-50.0,1,30,-1,0.0,0,
1,2020-09-02,inc,25.0,2,34,-1,0.0,0,
2,2020-09-02,exp,-50.0,5,30,-1,0.0,0,
3,2020-09-02,exp,-49.0,2,32,-1,0.0,0,mama chhoto
4,2020-09-02,inc,50.0,1,33,-1,0.0,0,mama chhoto
...,...,...,...,...,...,...,...,...,...
2249,2023-06-07,exp,-24.0,11,6,-1,0.0,0,canteen
2250,2023-06-08,exp,-146.0,11,11,-1,0.0,0,me+dk - ezee35+basmatiRice65+aloo10+haldi36
2251,2023-06-09,exp,-32.0,11,6,-1,0.0,0,canteen
2252,2023-06-10,exp,-20.0,11,6,-1,0.0,0,me+dk-fuchka


In [71]:
df.date_time = pd.to_datetime(df.date_time)
df.dtypes

date_time             datetime64[ns]
type                          object
amount                       float64
wallet_id                      int64
category_id                    int64
transfer_wallet_id             int64
trans_amount                 float64
subcategory_id                 int64
note                          object
period                        object
dtype: object

In [94]:
df['year'] = df.date_time.dt.strftime('%Y')
df['month'] = df.date_time.dt.strftime('%b')
df['period'] = df[['month', 'year']].agg('_'.join, axis=1)
df.period

0       Aug_2020
1       Sep_2020
2       Sep_2020
3       Sep_2020
4       Sep_2020
          ...   
2249    Jun_2023
2250    Jun_2023
2251    Jun_2023
2252    Jun_2023
2253    Jun_2023
Name: period, Length: 2254, dtype: object

In [125]:
df.shape[0]

2254

In [126]:
def insert_trans_from_df(df):
    """Inserts daily transaction"""
    for i in range(df.shape[0]):
        period = df.period.iloc[i]
        date = df.date_time.astype(str).iloc[i]
        _type = df.type.iloc[i]
        amount = df.amount.astype(float).iloc[i]
        wallet_id = df.wallet_id.astype(float).iloc[i]
        category_id = df.category_id.astype(float).iloc[i]
        transfer_wallet_id = df.transfer_wallet_id.astype(float).iloc[i]
        trans_amount = df.trans_amount.astype(float).iloc[i]
        subcategory_id = df.subcategory_id.astype(float).iloc[i]
        note = df.note.iloc[i]
        print({'key': period, 'date': date, 'type': _type, 'amount': amount, 'wallet_id': wallet_id, 'category_id': category_id, 'transfer_wallet_id': transfer_wallet_id, 'trans_amount': trans_amount, 'subcategory_id': subcategory_id, 'note': note})
        db2.put({'key': period, 'date': date, 'type': _type, 'amount': amount, 'wallet_id': wallet_id, 'category_id': category_id, 'transfer_wallet_id': transfer_wallet_id, 'trans_amount': trans_amount, 'subcategory_id': subcategory_id, 'note': note})

In [127]:
insert_trans_from_df(df)

{'key': 'Aug_2020', 'date': '2020-08-31', 'type': 'exp', 'amount': -50.0, 'wallet_id': 1.0, 'category_id': 30.0, 'transfer_wallet_id': -1.0, 'trans_amount': 0.0, 'subcategory_id': 0.0, 'note': ''}
{'key': 'Sep_2020', 'date': '2020-09-02', 'type': 'inc', 'amount': 25.0, 'wallet_id': 2.0, 'category_id': 34.0, 'transfer_wallet_id': -1.0, 'trans_amount': 0.0, 'subcategory_id': 0.0, 'note': ''}
{'key': 'Sep_2020', 'date': '2020-09-02', 'type': 'exp', 'amount': -50.0, 'wallet_id': 5.0, 'category_id': 30.0, 'transfer_wallet_id': -1.0, 'trans_amount': 0.0, 'subcategory_id': 0.0, 'note': ''}
{'key': 'Sep_2020', 'date': '2020-09-02', 'type': 'exp', 'amount': -49.0, 'wallet_id': 2.0, 'category_id': 32.0, 'transfer_wallet_id': -1.0, 'trans_amount': 0.0, 'subcategory_id': 0.0, 'note': 'mama chhoto'}
{'key': 'Sep_2020', 'date': '2020-09-02', 'type': 'inc', 'amount': 50.0, 'wallet_id': 1.0, 'category_id': 33.0, 'transfer_wallet_id': -1.0, 'trans_amount': 0.0, 'subcategory_id': 0.0, 'note': 'mama chho